In [1]:
import cv2
import albumentations as A
import numpy as np
import os
from pathlib import Path
import shutil
import random
from collections import defaultdict
import math

In [2]:
# --- Configuration ---
ORIGINAL_ROOT_PATH = Path(r"D:\VScodefiles\DeepLearningProject\RawImagesandLabels")
OUTPUT_DATASET_PATH = Path(r"D:\VScodefiles\DeepLearningProject\Augmented")

TARGET_TOTAL_IMAGES = 10000
ORIGINAL_NUM_IMAGES = 1170
AUGMENTED_NUM_IMAGES = TARGET_TOTAL_IMAGES - ORIGINAL_NUM_IMAGES

print(f"Original images: {ORIGINAL_NUM_IMAGES}")
print(f"Target total images: {TARGET_TOTAL_IMAGES}")
print(f"Images to be augmented: {AUGMENTED_NUM_IMAGES}")

Original images: 1170
Target total images: 10000
Images to be augmented: 8830


In [7]:
# --- Label Format and Class Definitions ---
CLASS_ID_TO_NAME = {0: "MILCO", 1: "NOMBO"}
CLASS_NAME_TO_ID = {v: k for k, v in CLASS_ID_TO_NAME.items()}

def parse_yolo_label_file(label_path):
    labels = []
    if not label_path.exists():
        print(f"Warning: Label file not found: {label_path} - Assuming background image.")
        return labels

    with open(label_path, 'r') as f:
        for line in f:
            parts = line.strip().split()
            if len(parts) == 5:
                class_id, x_center, y_center, width, height = map(float, parts)
                labels.append({
                    'class_id': int(class_id),
                    'x_center': x_center,
                    'y_center': y_center,
                    'width': width,
                    'height': height
                })
            else:
                print(f"Warning: Malformed line in {label_path}: {line.strip()}")
    return labels

def yolo_to_albumentations(annotations, image_width, image_height):
    """Convert YOLO format to normalized coordinates for Albumentations"""
    bboxes = []
    class_labels = []
    
    for ann in annotations:
        x_center = ann['x_center']
        y_center = ann['y_center']
        width = ann['width']
        height = ann['height']
        
        # Calculate normalized coordinates (already in 0-1 range)
        x_min = x_center - width / 2
        y_min = y_center - height / 2
        x_max = x_center + width / 2
        y_max = y_center + height / 2
        
        # Ensure coordinates are within [0,1] range
        x_min = max(0.0, min(1.0, x_min))
        y_min = max(0.0, min(1.0, y_min))
        x_max = max(0.0, min(1.0, x_max))
        y_max = max(0.0, min(1.0, y_max))
        
        bboxes.append([x_min, y_min, x_max, y_max])
        class_labels.append(ann['class_id'])
    
    return bboxes, class_labels

def albumentations_to_yolo(bboxes, class_labels, image_width, image_height):
    """Convert normalized coordinates back to YOLO format"""
    yolo_annotations = []
    
    for bbox, label in zip(bboxes, class_labels):
        x_min, y_min, x_max, y_max = bbox
        
        # Ensure coordinates are valid
        x_min = max(0.0, min(1.0, x_min))
        y_min = max(0.0, min(1.0, y_min))
        x_max = max(0.0, min(1.0, x_max))
        y_max = max(0.0, min(1.0, y_max))
        
        # Calculate YOLO format (normalized)
        x_center = (x_min + x_max) / 2.0
        y_center = (y_min + y_max) / 2.0
        width = x_max - x_min
        height = y_max - y_min
        
        # Ensure valid dimensions
        width = max(0.0, min(1.0, width))
        height = max(0.0, min(1.0, height))
        
        yolo_annotations.append({
            'class_id': label,
            'x_center': x_center,
            'y_center': y_center,
            'width': width,
            'height': height
        })
    
    return yolo_annotations

def analyze_class_distribution(image_label_pairs):
    """Analyze the distribution of classes in the dataset"""
    class_counts = defaultdict(int)
    images_with_class = defaultdict(list)
    
    for idx, (img_path, label_path) in enumerate(image_label_pairs):
        labels = parse_yolo_label_file(label_path)
        for label in labels:
            class_id = label['class_id']
            class_counts[class_id] += 1
            images_with_class[class_id].append(idx)
    
    # Count images with no objects (background)
    background_images = []
    for idx, (img_path, label_path) in enumerate(image_label_pairs):
        labels = parse_yolo_label_file(label_path)
        if len(labels) == 0:
            background_images.append(idx)
    
    print("\n--- Class Distribution Analysis ---")
    for class_id, count in class_counts.items():
        class_name = CLASS_ID_TO_NAME.get(class_id, f"Unknown_{class_id}")
        print(f"Class {class_id} ({class_name}): {count} instances in {len(images_with_class[class_id])} images")
    
    print(f"Background images (no objects): {len(background_images)}")
    
    return class_counts, images_with_class, background_images

def calculate_augmentation_weights(class_counts, images_with_class, total_images):
    """Calculate how many augmentations each class needs for balancing"""
    total_instances = sum(class_counts.values())
    if total_instances == 0:
        return {}
    
    # Calculate target instances per class (balanced)
    num_classes = len(class_counts)
    target_instances_per_class = total_instances * 2  # Aim for more balanced distribution
    
    augmentation_weights = {}
    for class_id, count in class_counts.items():
        shortage = max(0, target_instances_per_class - count)
        # Weight is proportional to how underrepresented the class is
        weight = shortage / count if count > 0 else 1.0
        augmentation_weights[class_id] = weight
    
    print("\n--- Augmentation Weights ---")
    for class_id, weight in augmentation_weights.items():
        class_name = CLASS_ID_TO_NAME.get(class_id, f"Unknown_{class_id}")
        print(f"Class {class_id} ({class_name}): weight = {weight:.2f}")
    
    return augmentation_weights

In [8]:
def get_augmentation_pipelines():
    """Define multiple augmentation strategies with corrected parameters"""
    
    pipelines = []
    
    # Pipeline 0: Basic Geometric + Noise (Balanced)
    pipelines.append(A.Compose([
        A.HorizontalFlip(p=0.5),
        A.Rotate(limit=15, p=0.4, border_mode=cv2.BORDER_CONSTANT),
        A.GaussNoise(p=0.6),
        A.RandomBrightnessContrast(brightness_limit=0.1, contrast_limit=0.1, p=0.3),
    ], bbox_params=A.BboxParams(format='albumentations', label_fields=['class_labels'])))

    # Pipeline 1: Sonar-Specific Augmentations
    pipelines.append(A.Compose([
        A.MultiplicativeNoise(multiplier=(0.8, 1.2), p=0.5),
        A.CLAHE(clip_limit=2.0, tile_grid_size=(8, 8), p=0.4),
        A.Blur(blur_limit=3, p=0.3),
        A.Affine(translate_percent=0.05, scale=(0.9, 1.1), rotate=(-10, 10), p=0.5),
    ], bbox_params=A.BboxParams(format='albumentations', label_fields=['class_labels'])))

    # Pipeline 2: Advanced Geometric Transformations
    pipelines.append(A.Compose([
        A.Rotate(limit=25, p=0.6, border_mode=cv2.BORDER_CONSTANT),
        A.Affine(shear=(-10, 10), p=0.3),
        A.Perspective(scale=(0.05, 0.1), p=0.3),
        A.RandomGamma(gamma_limit=(80, 120), p=0.4),
    ], bbox_params=A.BboxParams(format='albumentations', label_fields=['class_labels'])))

    # Pipeline 3: Environmental Variations (Sonar-like)
    pipelines.append(A.Compose([
        A.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.3, p=0.7),
        A.GaussNoise(p=0.5),
        A.ISONoise(color_shift=(0.01, 0.05), intensity=(0.1, 0.5), p=0.4),
        A.HueSaturationValue(hue_shift_limit=10, sat_shift_limit=20, val_shift_limit=10, p=0.3),
    ], bbox_params=A.BboxParams(format='albumentations', label_fields=['class_labels'])))
    
    return pipelines

def validate_bboxes(bboxes, class_labels):
    """Validate and fix bounding boxes before augmentation"""
    valid_bboxes = []
    valid_labels = []
    
    for bbox, label in zip(bboxes, class_labels):
        x_min, y_min, x_max, y_max = bbox
        
        # Skip invalid bboxes
        if x_min >= x_max or y_min >= y_max:
            continue
            
        # Ensure coordinates are within [0,1]
        x_min = max(0.0, min(1.0, x_min))
        y_min = max(0.0, min(1.0, y_min))
        x_max = max(0.0, min(1.0, x_max))
        y_max = max(0.0, min(1.0, y_max))
        
        # Skip if bbox is too small after validation
        if (x_max - x_min) < 0.01 or (y_max - y_min) < 0.01:
            continue
            
        valid_bboxes.append([x_min, y_min, x_max, y_max])
        valid_labels.append(label)
    
    return valid_bboxes, valid_labels

def apply_mosaic_augmentation(images, labels_list, image_size=(640, 640)):
    """Create mosaic augmentation by combining 4 images"""
    mosaic_img = np.zeros((image_size[0] * 2, image_size[1] * 2, 3), dtype=np.uint8)
    mosaic_bboxes = []
    mosaic_labels = []
    
    # Positions for the 4 images in the mosaic
    positions = [
        (0, 0),  # top-left
        (image_size[1], 0),  # top-right
        (0, image_size[0]),  # bottom-left
        (image_size[1], image_size[0])  # bottom-right
    ]
    
    for i, (img, labels) in enumerate(zip(images, labels_list)):
        if img is None:
            continue
            
        # Resize image to fit in mosaic quadrant
        img_resized = cv2.resize(img, image_size)
        x_offset, y_offset = positions[i]
        
        # Place image in mosaic
        mosaic_img[y_offset:y_offset+image_size[0], x_offset:x_offset+image_size[1]] = img_resized
        
        # Adjust bounding boxes for mosaic position and scaling
        orig_h, orig_w = img.shape[:2]
        scale_x = image_size[1] / orig_w
        scale_y = image_size[0] / orig_h
        
        for label in labels:
            # Convert YOLO to pixel coordinates
            x_center_pixel = label['x_center'] * orig_w
            y_center_pixel = label['y_center'] * orig_h
            width_pixel = label['width'] * orig_w
            height_pixel = label['height'] * orig_h
            
            # Calculate new coordinates in mosaic
            new_x_center = (x_center_pixel * scale_x + x_offset) / (image_size[1] * 2)
            new_y_center = (y_center_pixel * scale_y + y_offset) / (image_size[0] * 2)
            new_width = (width_pixel * scale_x) / (image_size[1] * 2)
            new_height = (height_pixel * scale_y) / (image_size[0] * 2)
            
            # Ensure valid coordinates
            if (0 <= new_x_center <= 1 and 0 <= new_y_center <= 1 and 
                0 < new_width <= 1 and 0 < new_height <= 1):
                mosaic_bboxes.append({
                    'class_id': label['class_id'],
                    'x_center': new_x_center,
                    'y_center': new_y_center,
                    'width': new_width,
                    'height': new_height
                })
                mosaic_labels.append(label['class_id'])
    
    return mosaic_img, mosaic_bboxes, mosaic_labels

def apply_cutmix_augmentation(image1, labels1, image2, labels2, beta=1.0):
    """Apply CutMix augmentation between two images"""
    lam = np.random.beta(beta, beta)
    lam = max(lam, 1 - lam)  # Ensure we use a significant portion from both
    
    h, w = image1.shape[:2]
    
    # Generate random cutting region
    cut_ratio = np.sqrt(1 - lam)
    cut_w = int(w * cut_ratio)
    cut_h = int(h * cut_ratio)
    
    cx = np.random.randint(w)
    cy = np.random.randint(h)
    
    x1 = max(0, cx - cut_w // 2)
    y1 = max(0, cy - cut_h // 2)
    x2 = min(w, cx + cut_w // 2)
    y2 = min(h, cy + cut_h // 2)
    
    # Create mixed image
    mixed_image = image1.copy()
    mixed_image[y1:y2, x1:x2] = image2[y1:y2, x1:x2]
    
    # Combine labels (all objects from both images)
    mixed_labels = labels1 + labels2
    
    return mixed_image, mixed_labels

In [9]:
# --- Helper Functions ---

def load_image_and_labels(image_path, label_path):
    image = cv2.imread(str(image_path))
    if image is None:
        raise FileNotFoundError(f"Could not load image: {image_path}")
    height, width = image.shape[:2]
    labels = parse_yolo_label_file(label_path)
    return image, labels, width, height

def save_yolo_annotation(annotation_list, output_label_path):
    with open(output_label_path, 'w') as f:
        for ann in annotation_list:
            f.write(f"{ann['class_id']} {ann['x_center']:.6f} {ann['y_center']:.6f} {ann['width']:.6f} {ann['height']:.6f}\n")

def apply_augmentation_pipeline(image, bboxes, class_labels, pipeline):
    """Apply a specific augmentation pipeline with proper error handling."""
    try:
        # Validate bboxes before augmentation
        valid_bboxes, valid_labels = validate_bboxes(bboxes, class_labels)
        
        if len(valid_bboxes) == 0:
            return image, bboxes, class_labels  # Return original if no valid bboxes
        
        transformed = pipeline(image=image, bboxes=valid_bboxes, class_labels=valid_labels)
        return transformed['image'], transformed['bboxes'], transformed['class_labels']
    except Exception as e:
        print(f"Augmentation error: {e}")
        return image, bboxes, class_labels  # Return original if augmentation fails

def copy_original_file(image_path, label_path, output_images_dir, output_labels_dir):
    output_img_path = output_images_dir / image_path.name
    shutil.copy2(image_path, output_img_path)
    output_lbl_path = output_labels_dir / label_path.name
    if label_path.exists():
        shutil.copy2(label_path, output_lbl_path)
    else:
        open(output_lbl_path, 'a').close()

def select_images_for_class_balancing(images_with_class, augmentation_weights, num_to_select):
    """Select images based on class balancing weights"""
    selected_indices = []
    
    for class_id, weight in augmentation_weights.items():
        if class_id in images_with_class and len(images_with_class[class_id]) > 0:
            num_for_class = int(num_to_select * weight)
            available_images = images_with_class[class_id]
            
            # Randomly select images containing this class
            selected = random.choices(available_images, k=min(num_for_class, len(available_images)))
            selected_indices.extend(selected)
    
    # Remove duplicates and ensure we don't exceed requested number
    selected_indices = list(set(selected_indices))
    if len(selected_indices) > num_to_select:
        selected_indices = random.sample(selected_indices, num_to_select)
    
    return selected_indices

In [10]:
# --- Main Execution ---

def main():
    # Collect all image-label pairs
    all_image_label_pairs = []
    for year_dir in ORIGINAL_ROOT_PATH.iterdir():
        if year_dir.is_dir():
            for img_file in year_dir.glob('*.jpg'):
                label_file = img_file.with_suffix('.txt')
                all_image_label_pairs.append((img_file, label_file))
    
    # If images are in a single folder
    if len(all_image_label_pairs) == 0:
        for img_file in ORIGINAL_ROOT_PATH.glob('*.jpg'):
            label_file = img_file.with_suffix('.txt')
            all_image_label_pairs.append((img_file, label_file))

    total_found = len(all_image_label_pairs)
    print(f"Found {total_found} image-label pairs.")

    if total_found != ORIGINAL_NUM_IMAGES:
        print(f"Warning: Found {total_found} files, expected {ORIGINAL_NUM_IMAGES}.")

    if AUGMENTED_NUM_IMAGES <= 0:
        print("Target number of images is less than or equal to the original number. No augmentation needed.")
        return

    # Analyze class distribution and calculate balancing weights
    class_counts, images_with_class, background_images = analyze_class_distribution(all_image_label_pairs)
    augmentation_weights = calculate_augmentation_weights(class_counts, images_with_class, total_found)

    # Get augmentation pipelines
    AUGMENTATION_PIPELINES = get_augmentation_pipelines()
    NUM_PIPELINES = len(AUGMENTATION_PIPELINES)
    print(f"\nUsing {NUM_PIPELINES} augmentation pipelines.")

    # Calculate augmented images per pipeline with class balancing
    base_aug_per_pipeline = AUGMENTED_NUM_IMAGES // NUM_PIPELINES
    remainder_aug = AUGMENTED_NUM_IMAGES % NUM_PIPELINES
    aug_counts_per_pipeline = [base_aug_per_pipeline] * NUM_PIPELINES
    for i in range(remainder_aug):
        aug_counts_per_pipeline[i] += 1

    print(f"\nAugmentation distribution:")
    for i, count in enumerate(aug_counts_per_pipeline):
        print(f"  Pipeline {i}: {count} augmented images")

    # Create output directories
    (OUTPUT_DATASET_PATH / "images").mkdir(parents=True, exist_ok=True)
    (OUTPUT_DATASET_PATH / "labels").mkdir(parents=True, exist_ok=True)

    # Copy original images
    print("\nCopying original images and labels...")
    for img_path, lbl_path in all_image_label_pairs:
        copy_original_file(img_path, lbl_path, OUTPUT_DATASET_PATH / "images", OUTPUT_DATASET_PATH / "labels")

    # Generate augmented images with class balancing
    print("\nStarting class-balanced augmentation...")
    
    # Track augmented images per class for monitoring
    augmented_class_counts = defaultdict(int)
    
    for pipeline_idx in range(NUM_PIPELINES):
        target_count = aug_counts_per_pipeline[pipeline_idx]
        pipeline = AUGMENTATION_PIPELINES[pipeline_idx]
        
        print(f"\nApplying Pipeline {pipeline_idx} to generate {target_count} images...")
        
        # Select images for this pipeline based on class balancing
        selected_indices = select_images_for_class_balancing(
            images_with_class, augmentation_weights, target_count
        )
        
        # If we need more images, add random selection
        if len(selected_indices) < target_count:
            additional_needed = target_count - len(selected_indices)
            all_indices = list(range(len(all_image_label_pairs)))
            additional_indices = random.choices(all_indices, k=additional_needed)
            selected_indices.extend(additional_indices)
        
        random.shuffle(selected_indices)
        
        aug_counter = 0
        for idx in selected_indices:
            if aug_counter >= target_count:
                break
                
            img_path, lbl_path = all_image_label_pairs[idx]
            
            try:
                image, labels, width, height = load_image_and_labels(img_path, lbl_path)
                
                # Convert to format for augmentation (normalized coordinates)
                bboxes_norm, class_labels = yolo_to_albumentations(labels, width, height)
                
                # Apply augmentation
                aug_image, aug_bboxes_norm, aug_class_labels = apply_augmentation_pipeline(
                    image.copy(), bboxes_norm, class_labels, pipeline
                )
                
                # Convert back to YOLO format
                aug_labels_yolo = albumentations_to_yolo(aug_bboxes_norm, aug_class_labels, width, height)
                
                # Update class counts for monitoring
                for label in aug_labels_yolo:
                    augmented_class_counts[label['class_id']] += 1
                
                # Save augmented image and labels
                stem = img_path.stem
                aug_image_name = f"{stem}_aug_p{pipeline_idx}_c{aug_counter:04d}{img_path.suffix}"
                aug_label_name = f"{stem}_aug_p{pipeline_idx}_c{aug_counter:04d}.txt"
                
                aug_image_path = OUTPUT_DATASET_PATH / "images" / aug_image_name
                cv2.imwrite(str(aug_image_path), aug_image)
                
                aug_label_path = OUTPUT_DATASET_PATH / "labels" / aug_label_name
                save_yolo_annotation(aug_labels_yolo, aug_label_path)
                
                aug_counter += 1
                
            except Exception as e:
                print(f"Error processing {img_path} with Pipeline {pipeline_idx}: {e}")
                continue

    # Apply advanced augmentations (Mosaic and CutMix)
    print("\nApplying advanced augmentations (Mosaic, CutMix)...")
    
    # Calculate how many advanced augmentations to create
    num_advanced_aug = min(500, AUGMENTED_NUM_IMAGES // 20)  # Reduced for stability
    
    mosaic_count = num_advanced_aug // 2
    cutmix_count = num_advanced_aug // 2
    
    print(f"Generating {mosaic_count} mosaic and {cutmix_count} CutMix augmentations...")
    
    # Mosaic Augmentation
    successful_mosaics = 0
    for i in range(mosaic_count):
        try:
            # Select 4 random images
            selected_indices = random.sample(range(len(all_image_label_pairs)), 4)
            images = []
            labels_list = []
            
            for idx in selected_indices:
                img_path, lbl_path = all_image_label_pairs[idx]
                image, labels, width, height = load_image_and_labels(img_path, lbl_path)
                images.append(image)
                labels_list.append(labels)
            
            # Apply mosaic
            mosaic_img, mosaic_labels, mosaic_class_labels = apply_mosaic_augmentation(images, labels_list)
            
            # Only save if we have valid objects
            if len(mosaic_labels) > 0:
                # Save mosaic
                mosaic_name = f"mosaic_{successful_mosaics:04d}.jpg"
                mosaic_label_name = f"mosaic_{successful_mosaics:04d}.txt"
                
                mosaic_path = OUTPUT_DATASET_PATH / "images" / mosaic_name
                cv2.imwrite(str(mosaic_path), mosaic_img)
                
                mosaic_label_path = OUTPUT_DATASET_PATH / "labels" / mosaic_label_name
                save_yolo_annotation(mosaic_labels, mosaic_label_path)
                
                # Update counts
                for label in mosaic_labels:
                    augmented_class_counts[label['class_id']] += 1
                
                successful_mosaics += 1
                
        except Exception as e:
            print(f"Error creating mosaic {i}: {e}")
            continue
    
    # CutMix Augmentation
    successful_cutmix = 0
    for i in range(cutmix_count):
        try:
            # Select 2 random images
            idx1, idx2 = random.sample(range(len(all_image_label_pairs)), 2)
            
            img_path1, lbl_path1 = all_image_label_pairs[idx1]
            img_path2, lbl_path2 = all_image_label_pairs[idx2]
            
            image1, labels1, w1, h1 = load_image_and_labels(img_path1, lbl_path1)
            image2, labels2, w2, h2 = load_image_and_labels(img_path2, lbl_path2)
            
            # Resize images to same size if different
            if image1.shape != image2.shape:
                image2 = cv2.resize(image2, (image1.shape[1], image1.shape[0]))
            
            # Apply CutMix
            cutmix_img, cutmix_labels = apply_cutmix_augmentation(image1, labels1, image2, labels2)
            
            # Save CutMix
            cutmix_name = f"cutmix_{successful_cutmix:04d}.jpg"
            cutmix_label_name = f"cutmix_{successful_cutmix:04d}.txt"
            
            cutmix_path = OUTPUT_DATASET_PATH / "images" / cutmix_name
            cv2.imwrite(str(cutmix_path), cutmix_img)
            
            cutmix_label_path = OUTPUT_DATASET_PATH / "labels" / cutmix_label_name
            save_yolo_annotation(cutmix_labels, cutmix_label_path)
            
            # Update counts
            for label in cutmix_labels:
                augmented_class_counts[label['class_id']] += 1
            
            successful_cutmix += 1
                
        except Exception as e:
            print(f"Error creating CutMix {i}: {e}")
            continue

    # Final statistics
    final_count = len(list((OUTPUT_DATASET_PATH / "images").glob('*')))
    print(f"\n--- Augmentation Complete ---")
    print(f"Final dataset contains {final_count} images in {OUTPUT_DATASET_PATH}")
    
    print("\n--- Class Distribution After Augmentation ---")
    for class_id in class_counts.keys():
        class_name = CLASS_ID_TO_NAME.get(class_id, f"Unknown_{class_id}")
        original_count = class_counts[class_id]
        augmented_count = augmented_class_counts[class_id]
        total_count = original_count + augmented_count
        print(f"Class {class_id} ({class_name}): {original_count} (original) + {augmented_count} (augmented) = {total_count} total")
    
    print("\nIMPORTANT: Class IDs in output labels:")
    for class_id, class_name in CLASS_ID_TO_NAME.items():
        print(f"  ID {class_id} -> {class_name}")
    print("Ensure your model training configuration uses: names = ['MILCO', 'NOMBO']")

if __name__ == "__main__":
    main()

Found 1170 image-label pairs.

--- Class Distribution Analysis ---
Class 0 (MILCO): 437 instances in 437 images
Class 1 (NOMBO): 231 instances in 231 images
Background images (no objects): 866

--- Augmentation Weights ---
Class 0 (MILCO): weight = 2.06
Class 1 (NOMBO): weight = 4.78

Using 4 augmentation pipelines.

Augmentation distribution:
  Pipeline 0: 2208 augmented images
  Pipeline 1: 2208 augmented images
  Pipeline 2: 2207 augmented images
  Pipeline 3: 2207 augmented images

Copying original images and labels...

Starting class-balanced augmentation...

Applying Pipeline 0 to generate 2208 images...

Applying Pipeline 1 to generate 2208 images...

Applying Pipeline 2 to generate 2207 images...

Applying Pipeline 3 to generate 2207 images...

Applying advanced augmentations (Mosaic, CutMix)...
Generating 220 mosaic and 220 CutMix augmentations...

--- Augmentation Complete ---
Final dataset contains 16225 images in D:\VScodefiles\DeepLearningProject\Augmented

--- Class Distr